# LSTM Modelling

On this Notebook, a LSTM Model will be tried for the Competitition Data.

## Libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style('darkgrid')
from cnr_methods import get_selected_features, transform_data, revert_data,metric_cnr, get_simplified_data
import random

from sklearn.model_selection import TimeSeriesSplit, train_test_split
import tensorflow as tf

## Read Data

Here, the data used correspond to the results of the Feature Engineering and Selection Step. For simplicity, during Hyperparameter Optimization, only Wind Farm 3 Training Data is used.

In [2]:
full_data = get_selected_features(0)

#full_data = full_data.rename({'Unnamed: 0' : 'Time'},axis=1)
full_data = full_data.set_index('Time')

full_label = pd.read_csv('Data/Y_train.csv')
X = full_data[full_data['Set']=='Train']

WF = 'WF3'
X = X[X['WF']==WF]
y = full_label[full_label['ID'].isin(X['ID'])]

In [3]:
X = X.drop(['ID','WF','Set'],axis=1)

In [4]:
X.head()

,U_100m,V_100m,U_10m,V_10m,T,CLCT,Wind Direction 100m,Wind Direction 10m,Wind Speed 100m,V_100m_lag_21_days,...,V_10m_Rolling_14_Window_Variance,U_100m_Rolling_14_Window_Mean,Wind Speed 10m,V_100m_Rolling_7_Window_Mean,T_lag_21_days,U_10m_Rolling_14_Window_Variance,CLCT_Rolling_7_Window_Mean,V_100m_lag_7_days,U_10m_lag_7_days,U_100m_lag_21_days
Time,,,,,,,,,,,,,,,,,,,,,
2018-05-01 01:00:00,5.789500,3.820200,1.054669,1.317597,275.690,86.504507,0.583268,0.895782,6.936299,NaN,...,NaN,NaN,1.687717,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 02:00:00,6.003300,3.920600,0.876879,1.483483,275.770,98.976088,0.578533,1.036951,7.170127,NaN,...,NaN,NaN,1.723264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 03:00:00,5.931829,0.907656,0.949640,1.419591,276.875,64.193607,0.151837,0.981212,6.000870,NaN,...,NaN,NaN,1.707938,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 04:00:00,5.205300,1.683800,1.027462,1.029786,275.650,57.482484,0.312855,0.786528,5.470862,NaN,...,NaN,NaN,1.454695,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 05:00:00,4.845900,0.702200,1.011645,0.785352,275.530,89.971463,0.143904,0.660129,4.896512,NaN,...,NaN,NaN,1.280704,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scaling Data

## Model

Model Creation Function:

In [5]:
def LSTM_Model(input_shape):
  # Numerical branch

  input_layer = tf.keras.layers.Input(input_shape)

  hidden_1 = tf.keras.layers.LSTM(units=256, activation="relu", kernel_initializer="he_normal")(input_layer)
  hidden_1 = tf.keras.layers.Dropout(rate=0.5)(hidden_1)
  hidden_1 = tf.keras.layers.BatchNormalization()(hidden_1)

  # Output

  outputs = tf.keras.layers.PReLU()(hidden_1)
  outputs = tf.keras.layers.Dropout(rate=0.5)(outputs)
  outputs = tf.keras.layers.BatchNormalization()(outputs)
  outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(outputs)

  model = tf.keras.Model(inputs=input_layer, outputs=outputs)

  return model

In [6]:
model = LSTM_Model((X.shape[1],1))

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 93, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
p_re_lu (PReLU)              (None, 256)               256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024  

## Validation

In [16]:
def metric_cnr(preds,labels):
    cape_cnr = 100*np.sum(np.abs(preds-labels))/np.sum(labels)
    return 'CAPE', cape_cnr

In [11]:
random.seed(317)
tf.random.set_seed(317)

patience = 10
epochs = 40
k_fold_splits = 5
total_it = 120
monitor = tf.keras.metrics.RootMeanSquaredError()
batch_size = 1 * ((len(X) - len(X) // k_fold_splits) // (total_it))

In [25]:
# Define Time Split Cross Validation
tscv = TimeSeriesSplit(n_splits=k_fold_splits)

# Separating Data from Hold Out Set

X_cv, _, y_cv, _ = train_test_split(X, y, test_size=0.125, shuffle=False)

train_scores = np.empty(0)
val_scores = np.empty(0)
test_scores = np.empty(0)
for train_index, test_index in tscv.split(X_cv):

    # Get the Data of the Split
    X_train, X_test = X_cv.iloc[train_index], X_cv.iloc[test_index]
    y_train, y_test = y_cv.iloc[train_index], y_cv.iloc[test_index]

    # Separating Training Set of Split on Train and Validation Subsets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.143, shuffle=False)

    # Callbacks
    callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=patience, verbose=0, min_delta=1e-8)]

    # Train the Model
    model.compile(optimizer='adam', loss='hinge', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    history = model.fit(x = X_train, y = y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, y_val), callbacks=callbacks_list)

    # Train and Validation Score
    #train_score = np.array(progress['train']['CAPE']).mean()
    #val_score = np.array(progress['eval']['CAPE']).mean()

    # Test Score
    preds = model.predict(X_test,batch_size = batch_size,callbacks=callbacks_list)
    test_score = metric_cnr(preds,y_test)

    #train_scores = np.append(train_scores,train_score)
    #val_scores = np.append(val_scores,val_score)
    test_scores = np.append(test_scores,test_score[1])

. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error
58/58 [==============================] - 10s 170ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 23/40
58/58 [==============================] - 10s 171ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 24/40
58/58 [==============================] - 10s 168ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 25/40
58/58 [==============================] - 10s 171ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 26/40
58/58 [==============================] - 10s 176ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan
Epoch 27/40
58/58 [==============================] - 10s 169ms/step - loss: nan - root_mean_squared_error: nan 

KeyboardInterrupt: 

In [21]:
history.history[]

{'loss': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 'accuracy': [0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607,
  0.020389843732118607],
 'val_loss': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 'val_accuracy': [0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897,
  0.016129031777381897]}

### Hold Out Score

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.125, shuffle=False)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.143, shuffle=False)